<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><strong>Описание данных</strong></a></span></li><li><span><a href="#Общая-информация" data-toc-modified-id="Общая-информация-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><strong>Общая информация</strong></a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><strong>Подготовка данных</strong></a></span><ul class="toc-item"><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Разбиение-на-тестовую-и-тренировочную-выборки" data-toc-modified-id="Разбиение-на-тестовую-и-тренировочную-выборки-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Разбиение на тестовую и тренировочную выборки</a></span></li><li><span><a href="#TF-IDF-столбцы" data-toc-modified-id="TF-IDF-столбцы-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>TF-IDF столбцы</a></span></li></ul></li><li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><strong>Обучение модели</strong></a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект: Машинное обучение для текстов с BERT

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 


**Цель исследования:**
1. Обучить модель классифицировать комментарии на позитивные и негативные со значением метрики качества *F1* не меньше 0.75.

**Ход исследования**

Данные получим из файла `/datasets/toxic_comments.csv`, 

просмотрим общую информацию о датасете, оценим качество данных.

Далее приступим к подготовке данных: лемматизируем текст, добавим новые столбцы. 

После этого применим пайплайн для подготовки признаков и обучения моделей.

Завершающим этапом исследования будет выбор лучшей модели и проверка на тестовой выборке.

Таким образом, иследование будет состоять из следующих этапов:

- [Обзор данных](#info)

- [Подготовка данных](#preprocessing)

- [Обучение модели](#model)

- [Итоговые выводы](#final)

## **Описание данных**

Данные находятся в файле `toxic_comments.csv`. 

Столбец **text** в нём содержит текст комментария, а **toxic** — целевой признак.

In [1]:
# Устанавливаем и обновляем необходимые библиотеки
!pip install -Uq scikit-learn 

In [2]:
# Импротируем необходимые библиотеки
import os
import pandas as pd
import numpy as np
import torch
import transformers
from numpy.random import RandomState
from tqdm import notebook



import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import f1_score
from transformers import BertTokenizer, BertModel



# Зафиксируем случайность  и размер тестовой выборки
RANDOM_STATE = 42
TEST_SIZE = 0.5
state = RandomState(12345)

sklearn.__version__


'1.7.2'

## **Общая информация**
<a id='info'></a>

In [3]:
# Загружаем файл
pth1 = '/datasets/toxic_comments.csv'
pth2 = 'toxic_comments.csv'
    
if os.path.exists(pth1):
    comments = pd.read_csv(pth1, index_col=0).reset_index(drop=True)
elif os.path.exists(pth2):
    comments = pd.read_csv(pth2, index_col=0).reset_index(drop=True)
else:
    print('Something is wrong')

In [4]:
# Посмотрим на первые 5 строк датасета и общую информацию
comments.info(), display(comments.shape, comments.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


(159292, 2)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


(None, None)

**Вывод:** Датафрейм содержит **159292 строки и 2 столбца**.

Текст представлен на английском языке.

В данных отсутсвуют пропуски, типы данных корректны.

## **Подготовка данных**
<a id='preprocessing'></a>

Обработка 159 292 текстов через BERT — это очень ресурсоёмко, поэтому ограничим датасет 4000 строками, сохраняя исходный баланс классов.

In [5]:
# Посчитаем долю каждого класса 
percent_0 = comments['toxic'].value_counts()[0]/comments['toxic'].count()
percent_1 = comments['toxic'].value_counts()[1]/comments['toxic'].count()
print(f'Доля класса "0": {percent_0}, доля класса "1": {percent_1}')

Доля класса "0": 0.8983878663084147, доля класса "1": 0.10161213369158527


Значит, для датасета из 4000 строк количество объектов класса "0" составит 3600, а класса "1" - 400 соответственно. Создаем датафрейм с учетом вычисленного баланса.

In [6]:
# Сэмплируем необходимое количество строк каждого класса
df_0 = comments[comments['toxic'] == 0].sample(n=3600, random_state=RANDOM_STATE)
df_1 = comments[comments['toxic'] == 1].sample(n=400, random_state=RANDOM_STATE)
# Объединяем и перемешиваем
df_small = pd.concat([df_0, df_1]).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

df_small.shape

(4000, 2)

### Токенизация

Разобьем текст на токены, преобразуем его в номера, применим метод padding и создадим маску для действительно важных токенов.  

In [7]:
# Инициализруем токенизатор и преобразуем текст
tokenizer = BertTokenizer.from_pretrained('./bert-base-uncased')
tokenized = df_small['text'].apply(lambda x: tokenizer.encode(x, max_length=512, truncation=True,
                                                              add_special_tokens=True))

In [8]:
# Применим padding к векторам
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
# Вектор с отступами
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values]) 
# Создадим маску для важных токенов
attention_mask = np.where(padded != 0, 1, 0)

### Эмбеддинги

Инициализируем BertModel.

In [9]:
model = BertModel.from_pretrained('./bert-base-uncased')

Преобразуем тексты в эмбеддинги.

In [10]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(len(padded)// batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    embeddings.append(batch_embeddings[0][:,0,:].numpy())
features = np.concatenate(embeddings)

  0%|          | 0/40 [00:00<?, ?it/s]

### Разбиение на тестовую и тренировочную выборки

Разобьем наш датасет на тренировочную и тестовую выборки в соотношении 50:50.

In [11]:
y = df_small['toxic']
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

**Вывод:** таким образом, мы разбили тексты на токены, преобразовали их в эмбеддинги, разбили датасет на тренировочную и тестовую выборки.

## **Обучение модели**
<a id='model'></a>

Создадим словарь для подбора лучших гиперпараметров.

In [12]:
param = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'class_weight': ['balanced']
}

Используем инструмент подбора гиперпараметров, который вернёт нам лучшую модель логистической регрессии.

В качестве метрики возьмем F1, так как имеем дело с бинарной классификацией. 

In [23]:
# Производим подбор гиперпараметров с помощью GridSearchCV
grid_search = GridSearchCV(
    LogisticRegression(max_iter=4000, random_state=RANDOM_STATE), 
    param, 
    cv=5,
    scoring='f1',
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

,estimator,LogisticRegre...ndom_state=42)
,param_grid,"{'C': [0.01, 0.1, ...], 'class_weight': [None, 'balanced'], 'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga']}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [24]:
print ('Метрика лучшей модели на кросс-валидации:', grid_search.best_score_)

Метрика лучшей модели на кросс-валидации: 0.6275073074878806


In [25]:
y_test_pred = grid_search.predict(X_test)
f1_score(y_test_pred, y_test)

0.6225895316804407

**Вывод:** таким образом, мы подобрали гиперпараметры для модели логистической регрессии, которые дали недостаточное значение метрики f1 - 0.62, что ниже порогового значения. 

### **Итоговые выводы**
<a id='final'></a>

Таким образом мы создали инструмент для интернет-магазина «Викишоп» :

1. Обучили модель классифицировать комментарии на позитивные и негативные.

Для этого были выполнены следующие действия:

- **Открыт файл** с данными и изучена общая информация: 

**`/datasets/toxic_comments.csv`** содержал **159292 строки и 2 столбца**.


В данных пропуски отсутствовали, типы данных были корректны. Мы имели дело с текстами на английском языке.


- Выполнена **подготовка данных:**
    
    токенизировали тексты и создали эмбеддинги.


- Обучили модель:

    перебирали гиперпараметры для моделей линейной регрессии

- Выбрали лучшую модель:

    **LogisticRegression(penalty='l2', C=1, solver='saga'), f1 = 0.62 на тестовой выборке.
    
    
**Рекомендации для бизнеса:** 

Использовать TF-IDF для подготовки данных и биграммы, чтобы улучшить метрику f1.

